In [1]:
import rpy2.robjects as robjects
import random
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score
from sklearn import metrics
import numpy as np
class ESCC:
    def __init__(self,path,k,n,p):
        self.path,self.k,self.n,self.p=path,k,n,p
        robjects.r('''
                    f <- function(path){
                    library('tidyverse')
                    a<-read_rds(path)
                    print(a)
                    tryCatch({
                        counts(a) <-assay(a, "normcounts")
                    }, warning = function(w) {
                    }, error = function(e) {
                    }, finally = {
                    })
                    data<-assay(a, "counts")
                    library(SC3)
                    t <- sc3_prepare(a)
                    gene_filter <- rowData(t)$sc3_gene_filter
                    t<-sc3_calc_dists(t)
                    t<-sc3_calc_transfs(t)
                    out<-metadata(t)$sc3$transformations
                    return(list(gene_filter,out,data))
                   }
                   ''')
        self.transformations()
        self.cluster=self.init_clusters()
    def transformations(self):
        gene_filter,out,logdata=robjects.r['f'](self.path)
        data=np.array(logdata)
        filtered=list()
        for i,b in enumerate(gene_filter):
            if b:
                filtered.append(data[i])
        filtered=np.array(filtered)
        X=np.array(np.log(filtered+1)).transpose()
        cell_num=X.shape[0]
        split_point=list(range(int(np.floor(0.04*cell_num)),int(np.ceil(0.07*cell_num)+1)))
        newX=np.array(out)
        import umap
        tx=list()
        m=['euclidean','chebyshev','correlation']
        for i in range(3):
            reducer = umap.UMAP(n_components=split_point[-1],metric=m[i])
            tx.append(reducer.fit_transform(X))
        self.newX=[elem[:,:split_point[-1]] for elem in newX]+tx
        self.space=list()
        for elem in split_point:
            self.space.extend([(i,elem) for i in range(len(self.newX))])
    def get_coord(self,x):
        return self.newX[x[0]][:,:x[1]]
    def init_clusters(self):
        cluster=list()
        for _ in range(self.n):
            clf = KMeans(self.k,init='k-means++',algorithm='full')
            coord = self.get_coord(random.choice(self.space))
            cluster.append(clf.fit(coord).labels_)
        return cluster
    def m_score(self,coord,c):
        try:
            return calinski_harabasz_score(coord,c)
        except:
            return 0
    def get_score(self,cluster,space):
        return np.array([[self.m_score(self.get_coord(sp),c) for sp in space] for c in cluster])
    def selection(self,score,n,num):
        l,S=[0]*n,[list()]*n
        tp,choice=list(),list()
        for i,plan1 in enumerate(score):
            for j,plan2 in enumerate(score):
                if all(plan1<plan2):
                    l[i]+=1
                if all(plan1>plan2):
                    S[i].append(j)
            if l[i]==0:
                tp.append(i)
        while(len(choice)+len(tp)<num):
            choice.extend(tp)
            t,tp=tp,list()
            for elem in t:
                for e in S[elem]:
                    l[e]-=1
                    if l[e]==0:
                        tp.append(e)
        print(len(tp),len(choice))
        n=num-len(choice)
        crowd=[0]*len(tp)
        for i in range(score.shape[1]):
            s=[score[j][i] for j in tp]
            c=(-1)/(max(s)-min(s)+1e-6)
            r=np.argsort(s)
            crowd[r[0]]=crowd[r[-1]]=-np.inf
            for j,elem in enumerate(r[1:-1]):
                crowd[elem]+=(score[tp[r[j+2]]][i]-score[tp[r[j]]][i])*c
        tp=np.array(tp,dtype=np.int)
        choice.extend(tp[np.argsort(crowd)[:n]])
        return choice
    def generate(self,cluster,tot):
        tp=list()
        for _ in range(tot):
            a,b,c=random.sample(cluster,3)
            na,nb,nc=max(a)+1,max(b)+1,max(c)+1
            l,visit=0,np.zeros((na,nb))-1
            t=np.zeros(len(c),dtype=np.int)
            for i in range(len(c)):
                x,y=a[i],b[i]
                if visit[x][y]==-1:
                    visit[x][y]=l
                    l+=1
                t[i]=visit[x][y]
            visit=np.zeros((l,nc))
            for i,j in zip(t,c):
                visit[i][j]+=1
            clu=[list() for _ in range(nc)]
            for i,elem in enumerate(visit):
                label=np.argmax(elem)
                clu[label].append(i)
            rand=list(range(nc))
            random.shuffle(rand)
            index1,rank,s,visit=list(),list(),0,[False]*(l+1)
            for i in rand:
                le=len(clu[i])
                if le==0:
                    continue
                s+=le
                index1.append(s)
                visit[s]=True
                random.shuffle(clu[i])
                rank.extend(clu[i])
            index1=index1[:-1]
            index2=list()
            for i in range(1,l):
                if visit[i]==False:
                    index2.append(i)
            n=self.k-1
            sele=list()
            sele.extend(random.sample(index1,min(len(index1),n)))
            sele.append(l)
            n-=len(index1)
            if n>0:
                sele.extend(random.sample(index2,n))
            label=[0]*l
            l=0
            sele=np.sort(sele)
            for i,j in enumerate(rank):
                if i>=sele[l]:
                    l+=1
                label[j]=l 
            for i,j in enumerate(t):
                t[i]=label[j]
            tp.append(t)
        return tp+cluster
    def evolve(self,cluster,tot,termination=True,m=50,n=10):
        b=True
        r=0
        re=list()
        while ((b or (not termination)) and r<tot):
            for _ in range(m if tot-r>=m else tot-r):
                selected=self.selection(self.get_score(cluster,random.sample(self.space,min(len(self.space),self.p))),self.n,self.n//2)
                cluster=self.generate([cluster[i] for i in selected],self.n-self.n//2)
            r+=m
            score=self.get_score(cluster,self.space)
            grade=-np.sum(score,axis=1)
            label=np.argsort(grade)
            s=0
            for elem1 in re:
                for elem2 in [cluster[i] for i in label[:n]]:
                    if metrics.adjusted_rand_score(elem1,elem2)>0.99:
                        s+=1
                        break
            if s>=n:
                b=False
            re=[cluster[i] for i in label[:n]]
        return cluster
    def res(self,b,tot,reuse=True):    
        cluster=self.evolve(self.init_clusters() if reuse==False else self.cluster,tot)
        self.cluster=cluster
        selected=cluster[self.n-self.n//2:]
        if b:
            import scipy
            from sklearn.preprocessing import normalize
            def Lap(K):
                tp=np.sum(K,1)
                D=np.mat(np.diag(tp))
                d=np.mat(np.diag(1/np.sqrt(tp)))
                L=d*(D-K)*d
                u,v=scipy.linalg.eigh(L)
                rank=np.argsort(u)
                return np.array(normalize(v.T[rank[1:]].real, norm='l2')).transpose()
            Lcoord=Lap([[metrics.adjusted_rand_score(elem1,elem2) for elem2 in selected] for elem1 in selected])
            rela=[scipy.stats.spearmanr(list(range(len(elem))),abs(elem))[0] for i,elem in enumerate(Lcoord)]
            labels=np.argsort(rela)
            select=[selected[i] for i in labels[np.argmax([rela[labels[j]]-rela[labels[j-1]] for j in range(1,self.n-self.n//2)])+1:]]
        else:
            select=selected
        cell_num=len(select[0])
        M=np.zeros((cell_num,cell_num),dtype=np.int)
        for elem in select:
            clu=list()
            visit=np.zeros((cell_num),dtype=np.int)-1
            l=0
            for i,e in enumerate(elem):
                if visit[e]==-1:
                    visit[e]=l
                    l+=1
                    tp=list()
                    tp.append(i)
                    clu.append(tp)
                else:
                    tp=clu[visit[e]]
                    tp.append(i)
                for j in tp:
                    M[i][j]+=1
                    M[j][i]+=1
                if i==j:
                    M[i][j]-=1
        from sklearn.cluster import AgglomerativeClustering
        ac = AgglomerativeClustering(n_clusters=self.k, affinity='precomputed',linkage='average')
        ac.fit(-M)
        return ac.labels_

In [ ]:
for i in range(100):
    c=ESCC('yan.rds',7,128,128)
    clu=c.res(False,300,True)
    with open('ESCC_Yan.txt', mode='a') as filename:
        for elem in clu:
            filename.write(str(elem))
            filename.write(',')
        filename.write('\n') 
    clu=c.res(True,0,True)
    with open('ESCC+_Yan.txt', mode='a') as filename:
        for elem in clu:
            filename.write(str(elem))
            filename.write(',')
        filename.write('\n') 